# hvplot

The core functionality provided by hvPlot is a **simple and high-level plotting interface** (API), modeled on [Pandas](https://pandas.pydata.org)'s `.plot` API and extended in various ways leveraging capabilities offered by the packages of the [HoloViz](https://holoviz.org) ecosystem, most notably [HoloViews](https://holoviews.org/). hvPlot can generate interactive plots with either [Bokeh](https://www.bokeh.org) (default) or [Plotly](https://plotly.com/python/), or static plots with [Matplotlib](https://matplotlib.org). hvPlot supports many data libraries of the Python ecosystem:

* [Pandas](https://pandas.pydata.org): DataFrame, Series (columnar/tabular data)
* [XArray](https://xarray.pydata.org): Dataset, DataArray (labelled multidimensional arrays)
* [GeoPandas](https://geopandas.org): GeoDataFrame (geometry data)
* [Rapids cuDF](https://docs.rapids.ai/api/cudf/stable/): GPU DataFrame, Series (columnar/tabular data)
* [Polars](https://www.pola.rs/): DataFrame, LazyFrame, Series (columnar/tabular data)
* [Dask](https://www.dask.org): DataFrame, Series (distributed/out of core arrays and columnar data)
* [Streamz](https://streamz.readthedocs.io): DataFrame(s), Series(s) (streaming columnar data)
* [Intake](https://github.com/ContinuumIO/intake): DataSource (data catalogues)
* [Ibis](https://ibis-project.org/): DataFrame interface for many backends (DuckDB, SQLite, SnowFlake, etc.)
* [NetworkX](https://networkx.github.io/documentation/stable/): Graph (network graphs)


## Register `.hvplot`

Let's create a simple Pandas DataFrame we'll plot later.

In [ ]:
import numpy as np
import pandas as pd
np.random.seed(1)

idx = pd.date_range('1/1/2000', periods=1000)
df = pd.DataFrame(np.random.randn(1000, 4), index=idx, columns=list('ABCD')).cumsum()
df.head(2)

The most convenient way to use hvPlot is to register the `.hvplot` *accessor* on the data type you are working with. This is done with a special import of the form `import hvplot.<data library>`.

In [ ]:
import hvplot.pandas  # noqa

In addition to registering the `.hvplot` accessor on Pandas objects (`DataFrame` and `Series`), the import above sets the Bokeh plotting library as the default one and loads its corresponding extension.

:::{note}
In a notebook, loading the extension means that there's actually some front-end code that is injected in the cell output of the import, this code being required for HoloViews plots to behave correctly. So make sure not to remove this cell!
:::

Now simply call `.hvplot()` on the DataFrame as you would call Pandas' `.plot()`.

In [ ]:
first_plot = df.hvplot()
first_plot

The same process can be applied to other libraries, we'll just show another example with Xarray.

In [ ]:
import hvplot.xarray  # noqa
import xarray as xr

air_ds = xr.tutorial.open_dataset('air_temperature').load()
air_ds['air'].isel(time=slice(0, 1000, 60)).hvplot.image(dynamic=False)

## Bokeh plots

As you can see, the default plots hvPlot generate are Bokeh plots. These plots are interactive and support panning, zooming, hovering, and clickable/selectable legends. It's worth spending some time getting used to interact with this kind of plot, try for instance zooming in on an axis and see what happens!

In [ ]:
first_plot

## `hvplot` namespace

The `.hvplot` namespace holds the range of supported plot methods (e.g. `line`, `scatter`, `hist`, etc.). Use tab completion to explore the available plot types.

```python
df.hvplot.<TAB>
```

Similarly to Panda's API, every plot method accepts a wide range of parameters. You can explore them by calling `hvplot.help('line')` or using tab completion:

```python
df.hvplot.line(<TAB>
```

## Compose plots 

The object returned by a `.hvplot.<type>()` call is a `HoloViews` object. 

In [ ]:
plot1 = df['A'].hvplot.area(alpha=0.2, color='red', width=300)
plot2 = df['B'].hvplot.line(width=300)
print(plot2)

`HoloViews` objects can easily composed using the `+` and `*` operators:

- `<plot1> + <plot2>` lays the plots out in a *row* container
- `(<plot1> + <plot2>).cols(1)` lays the plots out in a *column* container
- `<plot1> * <plot2>` overlays the plots

In [ ]:
plot1 + plot2

In [ ]:
plot1 * plot2

## Widgets-based exploration

In [ ]:
from bokeh.sampledata.penguins import data as df_penguins
df_penguins.head(2)

The `groupby` parameter allows to explore a dimension of the data using widgets.

In [ ]:
df_penguins.hvplot.scatter(x='bill_length_mm', y='bill_depth_mm', groupby=['island', 'sex'], dynamic=False)

## Display large data

In [ ]:
NUM = 1_000_000
dists = [
    pd.DataFrame(dict(x=np.random.normal(x, s, NUM), y=np.random.normal(y, s, NUM)))
     for x,  y,    s in [
       ( 5,  2, 0.20), 
       ( 2, -4, 0.10), 
       (-2, -3, 0.50), 
       (-5,  2, 1.00), 
       ( 0,  0, 3.00)]
]
df_large_data = pd.concat(dists, ignore_index=True)
len(df_large_data)

hvPlot provides multiple ways to display arbitrarily large datasets. The most versatile option depends on [Datashader](https://datashader.org/) (optional dependency) and simply consists of setting `rasterize` to `True`. The plot returned is an image, each pixel of that image being colorized based on the number of points it contains; which means that **all** the points contribute to the image. This plot is also dynamic, zooming in and out and panning leads to a recomputation of the image. Luckily, this all happens really fast! The plot below is generated with no less than 5 million points.

In [ ]:
df_large_data.hvplot.points('x', 'y', rasterize=True, cnorm='eq_hist', aspect=1, colorbar=False)

## Geographic plots

hvPlot can generate geographic plots and handle geographic data (e.g. GeoPandas DataFrame) thank to [GeoViews](https://geoviews.org/) (optional dependency).

In [ ]:
import cartopy.crs as crs

proj = crs.Orthographic(-90, 30)
air_ds.air.sel(time='2013-06-01 12:00').hvplot.quadmesh(
    'lon', 'lat', projection=proj, project=True,
    global_extent=True, cmap='viridis', coastline=True
)

## Matplotlib or Plotly plots

hvPlot offers the possibility to create [Matplotlib](https://matplotlib.org/) and [Plotly](https://plotly.com/) plots. Load the chosen plotting library with the `extension` function.

In [ ]:
hvplot.extension('matplotlib')

In [ ]:
air_ds.air.isel(time=slice(0, 9, 3)).hvplot.quadmesh(
    'lon', 'lat', projection=proj, project=True, global_extent=True, 
    cmap='viridis', rasterize=True, dynamic=False, coastline=True,
    xaxis=None, yaxis=None, width=370
)

## Conclusion

We have briefly explored some of the main features of `.hvplot()`. There's a lot more to it! For more information on using `.hvplot()` take a look at the [Bokeh Plotting Guide](../user_guide/Plotting.ipynb) and the various plotting guides available.